In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../resources/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [4]:
import implementations as imp
imp.least_squares_GD(y,tX,np.zeros(tX.shape[1]),20,0.7)

(array([-6.37930228e+126,  3.20951389e+126,  4.83666900e+126,
         1.86178365e+126, -6.00829278e+127, -6.10431901e+127,
        -6.00738245e+127,  1.50396509e+125,  8.62267799e+125,
         5.88898891e+126,  8.31642064e+124, -2.95754864e+124,
        -6.00790372e+127,  2.08940978e+126, -9.58480689e+122,
        -7.46914939e+122,  2.58258592e+126, -1.79458232e+123,
         2.56460694e+123,  2.01727388e+126, -1.04938868e+123,
         8.89038188e+126,  2.00320540e+124, -3.59259430e+127,
        -3.72490183e+127, -3.72493429e+127, -6.01619209e+127,
        -6.00782195e+127, -6.00782313e+127,  1.21699318e+126]),
 8.506588790558098e+262)

In [5]:
imp.least_squares(y,tX)

(array([ 8.03494312e-05, -7.20202273e-03, -6.05417273e-03, -5.47559065e-04,
        -1.93874700e-02,  4.73451621e-04, -2.60379054e-02,  3.25106300e-01,
        -3.80780282e-05, -2.72725004e+00, -2.21220140e-01,  9.50794091e-02,
         6.40351613e-02,  2.73550972e+00, -3.31801241e-04, -9.54325119e-04,
         2.74026646e+00, -5.34164892e-04,  9.73498581e-04,  3.69225052e-03,
         3.54487449e-04, -5.43344598e-04, -3.30448035e-01, -1.40800498e-03,
         8.31432888e-04,  1.02117272e-03, -1.68047416e-03, -5.83664818e-03,
        -1.11087997e-02,  2.72770997e+00]),
 0.33968680947709307)

In [6]:
imp.ridge_regression(y,tX,0.003)

(array([ 9.05917814e-05, -7.29992050e-03, -5.80087143e-03, -6.94660865e-04,
        -2.14113305e-02,  4.78606808e-04, -2.62144348e-02,  3.04921175e-01,
        -4.03931845e-05,  5.25494808e-03, -2.11080473e-01,  9.50323954e-02,
         6.50281362e-02,  3.03312278e-03, -3.37453779e-04, -9.90775210e-04,
         7.36770495e-03, -5.26760152e-04,  9.59230084e-04,  3.78441243e-03,
         3.57545368e-04, -5.60991720e-04, -3.06197134e-01, -1.17788634e-03,
         6.96388603e-04,  9.00367082e-04, -1.54352163e-03, -5.41822572e-03,
        -1.04797297e-02, -5.00915122e-03]),
 0.3397348031826314)

In [7]:
imp.least_squares_SGD(y,tX,np.zeros(tX.shape[1]),1,20,0.7)

(array([-2.81112679e+126, -4.59537083e+125, -2.07834151e+126,
        -7.25154894e+125,  2.72239695e+127,  2.72239695e+127,
         2.72239695e+127, -7.26517553e+124, -7.25154894e+125,
        -2.06427985e+126, -2.11741984e+124, -3.83424770e+124,
         2.72239695e+127, -1.16166497e+126, -4.37109647e+124,
        -7.59491671e+124, -9.02642128e+125, -6.09337388e+124,
         2.47168715e+124, -1.10203817e+125,  6.91908579e+124,
        -3.91687210e+126,  2.28320510e+090,  2.72239695e+127,
         2.72239695e+127,  2.72239695e+127,  2.72239695e+127,
         2.72239695e+127,  2.72239695e+127,  9.79746140e+091]),
 2.0476662283239264e+262)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)